In [1]:
#!pip install flask-cors

In [2]:
#!pip install Flask-Session


In [3]:
import pandas as pd
import json

In [4]:
# pip install spacy

In [5]:
# spacy.cli.download("en_core_web_sm")

In [6]:
#pip install symspellpy

In [7]:
#pip install textblob

In [8]:
#pip install flair

In [9]:
import time
import requests
from bs4 import BeautifulSoup
import urllib.request
from urllib.request import Request, urlopen
from random import randint
from time import sleep
import multiprocess as mp
from multiprocess.dummy import Pool
import tqdm as tqdm
import numpy as np
import pandas as pd
from requests.exceptions import SSLError,ChunkedEncodingError
import re
from datetime import datetime as dt
import pysolr
from textblob import TextBlob
import flair
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import spacy
import re
import nltk
nltk.download('vader_lexicon')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
stopword = stopwords.words('english')
import ipywidgets as widgets

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/waichung/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [10]:
def get_latest_date(solr):
    connection = urllib.request.urlopen('http://ec2-3-132-215-195.us-east-2.compute.amazonaws.com:8983/solr/admin/cores?wt=xml')
    response = list(connection)
    last_mod = [str(x) for x in response if 'lastModified' in str(x)][0]
    result = re.search('%s(.*)%s' % ('>', '<'), last_mod).group(1)
    dt_obj=dt.strptime(result,"%Y-%m-%dT%H:%M:%S.%fZ")
    return dt_obj

def random_proxy(proxies):
    return proxies[randint(0, len(proxies) - 1)]
my_headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36",  
              "Accept":"text/html,application/xhtml+xml,application/xml; q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3", 
              "Accept-Encoding": "gzip", 
              "Accept-Language": "en-US,en;q=0.9,es;q=0.8", 
              "Upgrade-Insecure-Requests": "1",}
def get_proxies():
    proxies = []
    proxies_req = Request('https://www.sslproxies.org/')
    proxies_req.add_header('User-Agent', my_headers['User-Agent'])
    proxies_doc = urlopen(proxies_req).read().decode('utf8')
    proxies_list = str(BeautifulSoup(proxies_doc, 'html.parser').find_all("textarea", class_="form-control")[0]).split('\n')[3:-1]
    return proxies_list

def update_url_list(prox,latest_date):
    def get_urls(number):
        page_urls=[]
        date = []
        author =[]
        url = 'https://www.fool.ca/recent-headlines/page/' + str(number) + '/'
        number+=1
        proxy = random_proxy(prox)
        session = requests.Session()
        session.proxies = {'http': 'http://'+proxy}
        page = session.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        soup = BeautifulSoup(page.content, "html.parser")
        for a in soup.find("main").findAll('a', href=True):
            for c in a.findAll("time" ):
                b=c['datetime'][:-6]
                d=dt.strptime(b,"%Y-%m-%dT%H:%M:%S")  
                if d>latest_date:
                    if 'page' not in str(a['href']):
                        page_urls.append([(a['href']),b+'Z'])
                else:
                    number=-1
        return page_urls,number

    counter=0
    allurl=[]
    while counter>=0:
        page_url,counter=get_urls(counter)
        allurl.append(page_url)
    all_url=[item for sublist in allurl for item in sublist]
    return all_url

def update_content_df(prox,all_url):
    def get_info(url_lst):
        url,time=url_lst
        my_headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36",  
                  "Accept":"text/html,application/xhtml+xml,application/xml; q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3", 
                  "Accept-Encoding": "gzip", 
                  "Accept-Language": "en-US,en;q=0.9,es;q=0.8", 
                  "Upgrade-Insecure-Requests": "1",}
        counter = 0
        #url = 'https://www.fool.com.au/2022/03/05/5-asx-shares-making-big-news-this-week/'
        response = Request(url , headers=my_headers)
        proxy = random_proxy(prox)
        session = requests.Session()
        session.proxies = {'http': 'http://'+proxy}
        while True:
            try:
                page = session.get(url, headers=my_headers)
                break
            except (SSLError, ConnectionResetError, ChunkedEncodingError):
                proxy = random_proxy(prox)
                session = requests.Session()
                session.proxies = {'http': 'http://'+proxy}
                counter+=1
                if counter==10:
                    break
            except:
                break

        #             page = session.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        soup = BeautifulSoup(page.content, "html.parser")
        stocks_mentioned = soup.find_all(class_="chicklets")
        if len(stocks_mentioned)>0:
            stocks_mentioned = [i.text for i in stocks_mentioned[0].find_all('a')]
        key_points = soup.find_all(id='key-points')
        if len(key_points)>0:
            key_points = [i.text for i in key_points[0].find_all('li')]
        info={"title":[i.text for i in soup.findAll('title')][0],
              "author":[i.text for i in soup.findAll(class_='url fn n')][0],
              'date_published':url.split('/')[3]+'/'+url.split('/')[4]+'/'+url.split('/')[5],
              "content":[a.text for a in soup.find_all(class_='entry-content')[0].find_all('p')], 
              "stocks_mentioned": stocks_mentioned,
              'key_points':key_points,
             'url':url} 
    #     time.sleep(5)
        return info

    pool = Pool(processes=8)
    mapped_values = list(tqdm.tqdm(pool.imap_unordered(get_info, all_url), total=len(all_url)))
    df2=pd.DataFrame(mapped_values)
    return df2

class SentimentAnalysis():
    def __init__(self, list_of_sentences):
        self.list_of_sentences = list_of_sentences
        self.sentiment_vader = SentimentIntensityAnalyzer()
        self.sentiment_textblob = TextBlob
        self.sentiment_flair = flair.models.TextClassifier.load('en-sentiment')
    def get_vader_sentiment(self, sentence):
        try:
            self.sentiment_vader.polarity_scores(sentence)['compound']
        except:
            print(sentence)
        return self.sentiment_vader.polarity_scores(sentence)['compound']
    def get_textblob_sentiment(self, sentence):
        result = self.sentiment_textblob(sentence).sentiment
        return ((4-result.subjectivity)/4) * result.polarity
    def get_flair_sentiment(self, sentence):
        result = flair.data.Sentence(sentence)
        self.sentiment_flair.predict(result)
        return result.labels[0].score*(-1 if result.labels[0].value == 'NEGATIVE' else 1)
    def get_average_sentiment(self, sentence):
        return (self.get_vader_sentiment(sentence) + self.get_textblob_sentiment(sentence) + self.get_flair_sentiment(sentence))/3
    def get_weighted_sentiment(self, sentence):
        vader = self.get_vader_sentiment(sentence)
        vader_positive = vader>0.0
        textblob = self.get_textblob_sentiment(sentence)
        textblob_positive = textblob>0.0
        flair = self.get_flair_sentiment(sentence)
        flair_positive = flair>0.0
        if (vader==0.0 or textblob==0.0 or flair==0.0):
            # assumption that only one model gives us 0.0 prediction
            return (self.get_vader_sentiment(sentence) + self.get_textblob_sentiment(sentence) + self.get_flair_sentiment(sentence))/2
        # a bit difficult to understand, but the aim here is to remove the prediction that is of a different polarity
        # from the rest (e.g. only take the average of two positive predictions and discount the negative prediction)
        elif (vader_positive!=textblob_positive or vader_positive!=flair_positive):
            if vader_positive:
                if textblob_positive:
                    return (self.get_vader_sentiment(sentence) + self.get_textblob_sentiment(sentence))/2
                else:
                    return (self.get_vader_sentiment(sentence) + self.get_flair_sentiment(sentence))/2
            else:
                return (self.get_textblob_sentiment(sentence) + self.get_flair_sentiment(sentence))/2
        else:
            return (self.get_vader_sentiment(sentence) + self.get_textblob_sentiment(sentence) + self.get_flair_sentiment(sentence))/3
    def get_all_vadar_sentiment(self):
        '''returns all VADER (Valence Aware Dictionary and sEntiment Reasoner) sentiments, which is based off a form of lexicon and rule-based feeling analysis'''
        return self.get_all('vader')
    def get_all_textblob_sentiment(self):
        '''returns all TextBlob sentiments, which is based off noun-phrase extraction, position tracking and more'''
        return self.get_all('textblob')
    def get_all_flair_sentiment(self):
        '''returns all Flair sentiments, which is based off a pre-trained machine learning model from Twitter'''
        return self.get_all('flair')
    def get_all_average_sentiment(self):
        '''returns all average of VADER, TextBlob and Flair sentiments'''
        return self.get_all('average')
    def get_all_weighted_sentiment(self):
        '''returns all weighted average of VADER, TextBlob and Flair sentiments when there are conflicting non-zero sentiments'''
        return self.get_all('weighted')
    def get_all(self, which):
        '''please do not use this, this is the helper function'''
        scores = []
        if which == 'vader':
            for sentence in self.list_of_sentences:
                scores += [round(self.get_vader_sentiment(sentence),5)]
        elif which == 'textblob':
            for sentence in self.list_of_sentences:
                scores += [round(self.get_textblob_sentiment(sentence),5)]
        elif which == 'flair':
            for sentence in self.list_of_sentences:
                scores += [round(self.get_flair_sentiment(sentence),5)]
        elif which == 'average':
            for sentence in self.list_of_sentences:
                scores += [round(self.get_average_sentiment(sentence),5)]
        elif which == 'weighted':
            for sentence in self.list_of_sentences:
                scores += [round(self.get_weighted_sentiment(sentence),5)]
        return scores
    def change_sentences(self,list_of_sentences):
        '''enables changing of object's list of sentences on the fly'''
        self.list_of_sentences = list_of_sentences
        return

def get_preprocessed_df(dff):
    def convertColons(x):
        ex, tick = x.split(':')
        if ex=='TSX':
            return tick+'.TRT'
        elif ex=='TSXV':
            return tick+'.TRV'
        else:
            return tick

    def removeStockDuplicates(x):
        x = list(set(x))
        to_remove = [i.split('.')[0] for i in x if '.' in i]
        x = [i for i in x if i not in to_remove]
        return sorted(x)
    print('Preprocessing Unindexed Data')
    # data cleaning
    dff['title'] = dff['title'].apply(lambda x: x.split('|')[0])
    dff['title'] = dff['title'].apply(lambda x: x.replace('’',"'"))
    dff['title'] = dff['title'].apply(lambda x: x.replace('”','"'))
    dff['title'] = dff['title'].apply(lambda x: x.replace('“','"'))
    dff['title'] = dff['title'].astype('str')

    dff['content'] = dff['content'].apply(lambda x: ' '.join(x))
    dff['content'] = dff['content'].apply(lambda x: x.replace('\\xa0',' '))
    dff['content'] = dff['content'].apply(lambda x: x.replace('’',"'"))
    dff['content'] = dff['content'].apply(lambda x: x.replace('”','"'))
    dff['content'] = dff['content'].apply(lambda x: x.replace('“','"'))

    dff['stocks_mentioned'] = dff['stocks_mentioned'].apply(lambda x: str(x).replace('\\xa0',' '))
    dff['stocks_mentioned'] = dff['stocks_mentioned'].apply(lambda x: x.replace('’',"'"))
    dff['stocks_mentioned'] = dff['stocks_mentioned'].apply(lambda x: x.replace('”','"'))
    dff['stocks_mentioned'] = dff['stocks_mentioned'].apply(lambda x: x.replace('“','"'))
    dff['stocks_mentioned'] = dff['stocks_mentioned'].str[2:-2].str.split("', '").tolist()
    dff['stocks_mentioned'] = dff['stocks_mentioned'].apply(lambda x: list(set(x)))
    dff['stock_mentioned_in_content'] = dff['content'].apply(lambda x: [i for i in re.findall('\((.*?\:.*?)\)', x) if len(i)<=10])
    dff['stock_mentioned_in_content'] = dff['stock_mentioned_in_content'].apply(lambda x: [convertColons(i) for i in x])
    dff['stocks_mentioned'] = dff['stock_mentioned_in_content']+dff['stocks_mentioned']
    dff=dff.drop(['stock_mentioned_in_content'], axis=1)
    dff['stocks_mentioned'] = dff['stocks_mentioned'].apply(lambda x: removeStockDuplicates(x))
    dff['stocks_mentioned']=dff['stocks_mentioned'].apply(lambda x: str(x))
    
    dff['key_points'] = dff['key_points'].apply(lambda x: ' '.join(str(x)[2:-2].split("', '")))
    dff['key_points'] = dff['key_points'].apply(lambda x: str(x).replace('\\xa0',' '))
    dff['key_points'] = dff['key_points'].apply(lambda x: x.replace('’',"'"))
    dff['key_points'] = dff['key_points'].apply(lambda x: x.replace('”','"'))
    dff['key_points'] = dff['key_points'].apply(lambda x: x.replace('“','"'))
    dff['key_points'] = dff['key_points'].apply(lambda x: "NaN")
    
    summ=list(dff['content'])
    def get_summary(idx):
        text=summ[idx]
        words = word_tokenize(text)
        freqTable = dict()
        for word in words:
            word = word.lower()
            if word in stopword:
                continue
            if word in freqTable:
                freqTable[word]+=1
            else:
                freqTable[word]=1

        sentences = sent_tokenize(text)
        sentenceValue = dict()
        for sentence in sentences:
        #     print(sentence)
        #     word = word.lower()
            for word, freq in freqTable.items():
                if word in sentence.lower():  
                    if sentence in sentenceValue:
                        sentenceValue[sentence]+= freq
                    else:
                        sentenceValue[sentence]=freq
        sumValues = 0
        for sentence in sentenceValue:
            sumValues += sentenceValue[sentence]
        average = int(sumValues / len(sentenceValue))
        summary=""
        for sentence in sentences:
            if (sentence in sentenceValue) and (sentenceValue[sentence]>1.2*average):
                summary += " " + sentence
        if summary == "":
            summary=sentences[0]
        summ[idx]=summary
    pool = Pool(processes=8)
    print('Summarizing Text')
    mapped_summ_values = list(tqdm.tqdm(pool.imap_unordered(get_summary, list(range(len(summ)))), total=len(summ)))
    dff['extract_summarizer']=summ
    dff['extract_summarizer'] = dff['extract_summarizer'].apply(lambda x: str(x).replace('\\xa0',' '))
    dff['extract_summarizer'] = dff['extract_summarizer'].apply(lambda x: x.replace("', '",' ')) # ', ' -> ' '
    dff['extract_summarizer'] = dff['extract_summarizer'].apply(lambda x: x.replace("\", '",' ')) # ", ' -> ' '
    dff['extract_summarizer'] = dff['extract_summarizer'].apply(lambda x: x.replace("', \"",' ')) # ", ' -> ' '
    dff['extract_summarizer'] = dff['extract_summarizer'].apply(lambda x: x.replace("[",'')) # [ ->
    dff['extract_summarizer'] = dff['extract_summarizer'].apply(lambda x: x.replace("]",'')) # ] ->
    dff['extract_summarizer'] = dff['extract_summarizer'].apply(lambda x: x.replace('’',"'")) # ’ -> '
    dff['extract_summarizer'] = dff['extract_summarizer'].apply(lambda x: x.replace('”','"')) # ” -> "
    dff['extract_summarizer'] = dff['extract_summarizer'].apply(lambda x: x.replace('“','"')) # “ -> "
    
    
    title=list(dff['title'])
    content=list(dff['content'])
    nlp = spacy.load('en_core_web_sm')
    def lem_text_t(idx):
        x=title[idx]
        #print(x)
        try:
            doc = x.replace('TSX:', 'TSX')
            doc = nlp(re.sub(r'[^\w\s]', '', doc))
            text = []
            for q in doc:
                if q.lemma_ != '-PRON-':
                    text+=[q.lemma_]
            doc = ' '.join(text)
            title[idx]=doc.replace(' s ', ' ')
        except:
            title[idx]= ''
    def lem_text_c(idx):
        x=content[idx]
        try:
            doc = x.replace('TSX:', 'TSX')
            doc = nlp(re.sub(r'[^\w\s]', '', doc))
            text = []
            for q in doc:
                if q.lemma_ != '-PRON-':
                    text+=[q.lemma_]
            doc = ' '.join(text)
            content[idx]=doc.replace(' s ', ' ')
        except:
            content[idx]= ''
    test = SentimentAnalysis(['this is such a goddamn hassle', 'man... this is one of the best stocks to invest', 'what the hell', "damn I don't hate this as much as i thought"])
    dff['merged_key_points'] = dff.apply(lambda x: x['extract_summarizer'], axis=1)
    kp=list(dff['extract_summarizer'])
    def get_pred(idx):
        y=[kp[idx]]
        test.change_sentences(y)
        prediction_results = test.get_all_weighted_sentiment()
        kp[idx] = prediction_results[0]
    print('Lemmatizing Title Text')
    mapped_title_values = list(tqdm.tqdm(pool.imap_unordered(lem_text_t, list(range(len(title)))), total=len(title)))
    print('Lemmatizing Title Content')
    mapped_content_values = list(tqdm.tqdm(pool.imap_unordered(lem_text_c, list(range(len(content)))), total=len(content)))
    print('Analyzing Text Sentiment')
    mapped_sent_values = list(tqdm.tqdm(pool.imap_unordered(get_pred, list(range(len(kp)))), total=len(kp)))
    dff['lemmatized_title'] = title
    dff['lemmatized_content'] = content
    dff['predict'] = kp
    dff['predict_cat'] = dff['predict'].apply(lambda x: 'Positive' if x>0.2 else ('Negative' if x<-0.2  else 'Neutral'))
    
    return dff
def wrapper(a):
    solr = pysolr.Solr('http://ec2-3-132-215-195.us-east-2.compute.amazonaws.com:8983/solr/articleindex', always_commit=True)
    latest_date= get_latest_date(solr)
    #latest_date=dt.strptime("2022/3/26 10:00:00","%Y/%m/%d %H:%M:%S")
    s=latest_date.strftime("%d %B %Y")
    prox = get_proxies()
    print('Getting All Unindexed Article URLS from '+s)
    all_url = update_url_list(prox,latest_date)
    print('Getting All Unindexed Article Content')
    df= update_content_df(prox,all_url) 
    df1= get_preprocessed_df(df)
    print('Adding Unindexed Articles to Solr Index')
    js = df1.to_dict(orient='records')
    solr.add(js)
    return df1,js

### Solr via pysolr

In [11]:
import pysolr
from flask import Flask
from flask_cors import CORS
from flask_session import Session
from flask import request
from collections import OrderedDict
import json
from datetime import datetime
import pandas as pd
from symspellpy import SymSpell, Verbosity 
import pkg_resources
from nltk.corpus import stopwords 
import string 
# import spacy
# import re
# nlp = spacy.load('en_core_web_sm')

In [12]:
#this is for lemmatisation
import spacy
import re
nlp = spacy.load('en_core_web_sm')

In [13]:
solr = pysolr.Solr('http://ec2-3-132-215-195.us-east-2.compute.amazonaws.com:8983/solr/articleindex', always_commit=True)

In [14]:
# Check health of system, whether solr is online
solr.ping()

'{\n  "responseHeader":{\n    "zkConnected":null,\n    "status":0,\n    "QTime":0,\n    "params":{\n      "q":"{!lucene}*:*",\n      "distrib":"false",\n      "df":"_text_",\n      "rows":"10",\n      "echoParams":"all",\n      "rid":"solr-node-1-23"}},\n  "status":"OK"}\n'

The below link is a guide for us to phrase queries: <br>
https://solr.apache.org/guide/6_6/common-query-parameters.html

In [15]:
#!pip show Flask

# This Cell must be ran for the solr server to be deployed

In [ ]:
app = Flask(__name__)
CORS(app)
results =[]

def convertToDic(results):
    
    query_dic ={}

    for index in range(0,len(results)):
        
        query_dic[index]=results.docs[index]
    
    return query_dic

def sortDicbyDate(results):
    query_dic ={}

    for index in range(0,len(results)):
        
        query_dic[index]=results.docs[index]
    #sorting the dictionary
    ordered_dic = OrderedDict(reversed(sorted(query_dic.items(), key = lambda tup: datetime.strptime(tup[1]["date_published"][0], '%Y/%m/%d'))))
    #reordering the dictionary
    ordered_dic={i:v for i,(k,v) in enumerate(ordered_dic.items(), 0)}
    return ordered_dic

def sortCombinedDictByDate(query_dic):
    ordered_dic = OrderedDict(reversed(sorted(query_dic.items(), key = lambda tup: datetime.strptime(tup[1]["date_published"][0], '%Y/%m/%d'))))
    #reordering the dictionary
    ordered_dic={i:v for i,(k,v) in enumerate(ordered_dic.items(), 0)}
    return ordered_dic

title = 'title:""'

def getInputLength(input):
    inputlength=len(input.split())
    return inputlength

def generate_N_grams(text,ngram):
    words=[word for word in text.split(" ") ]  
    temp=zip(*[words[i:] for i in range(0,ngram)])
    ans=[' '.join(ngram) for ngram in temp]
    #print('ngram list is', ans)
    return ans

def querysyntax(userinput):
    return title[:-1]+userinput+title[-1:]
#     print(querystring)


def DictListUpdate(lis1, lis2):
    for aLis1 in lis1:
        if aLis1 not in lis2:
            lis2.append(aLis1)
    return lis2

def dictcombine(d1,d2):
    count=len(d1)
    for key,value in d2.items():
        if value not in d1.values():
            d1[count] = value
            count+=1

    return d1

def searchfunction(query):
    result = solr.search(f'{querysyntax(query)}',fq=[],rows=100,fl=['title','stocks_mentioned','key_points','date_published','url','predict_cat','extract_summarizer'])
    return result



def twosearch(x,strlen):
    query=generate_N_grams(x,strlen-1)
    results=searchfunction(query[0])
    results2=searchfunction(query[1])
    debug = convertToDic(results)
    debug2 = convertToDic(results2)
    print('two search ran')
    return(dictcombine(debug,debug2))
    

def threesearch(x,strlen):
    print('three two search running')
    results=twosearch(x,strlen)
    if len(results)>=10:
        return results 
    else:
        query = generate_N_grams(x,strlen-2)
        results=searchfunction(query[0])
        results2=searchfunction(query[1])
        results3=searchfunction(query[2])
        debug = convertToDic(results)
        debug2 = convertToDic(results2)
        debug3= convertToDic(results3)
        print('tthree search ran')
        return(dictcombine(dictcombine(debug,debug2),debug3))
    
        

def foursearch(x,strlen):
    results = threesearch(x,strlen)
    if len(results) >=10:
        return results
    else:
        query = generate_N_grams(x,strlen-3)
        results=searchfunction(query[0])
        results2=searchfunction(query[1])
        results3=searchfunction(query[2])            
        results4=searchfunction(query[3])
        debug = convertToDic(results)
        debug2 = convertToDic(results2)     
        debug3 = convertToDic(results3)  
        debug4 = convertToDic(results4) 
        return(dictcombine(dictcombine(dictcombine(debug,debug2),debug3),debug4))
        


def fivesearch(x,strlen):
    results=foursearch(x,strlen)
    if len(results)>=10:
        return results
    else:
        
        query = generate_N_grams(x,strlen-4)
        results=searchfunction(query[0])
        results2=searchfunction(query[1])
        results3=searchfunction(query[2])            
        results4=searchfunction(query[3])
        results5=searchfunction(query[4])
        debug = convertToDic(results)
        debug2 = convertToDic(results2)     
        debug3 = convertToDic(results3)  
        debug4 = convertToDic(results4) 
        debug5 = convertToDic(results5) 
        return(dictcombine(dictcombine(dictcombine(dictcombine(debug,debug2),debug3),debug4),debug5)) 

    
def lem_text(doc): 
    try: 
        doc = nlp(re.sub(r'[^\w\s]', '', doc)) 
        text = [] 
        for q in doc: 
            if q.lemma_ != '-PRON-': 
                text+=[q.lemma_] 
        doc = ' '.join(text)
        print('something')
        return doc.replace(' s ', ' ') 
    except: 
        print(x) 
        return ''
    
def remove_stopwords(sentence): 
    return ' '.join([word for word in sentence.split(' ') if word not in list(set(stopwords.words('english')))])

 
class spellchecker(): 
    def __init__(self): 
        self.speller = SymSpell(max_dictionary_edit_distance=3, prefix_length=7) 
        dictionary_path = pkg_resources.resource_filename( 
            "symspellpy", "frequency_dictionary_en_82_765.txt" 
        ) 
        self.speller.load_dictionary(dictionary_path, term_index=0, count_index=1)
        
        df = pd.read_csv('https://www.tsx.com/files/trading/interlisted-companies.txt', delimiter='\t', header=1).iloc[:,0:2] 
        df['Symbol'] = df['Symbol'].apply(lambda x: [x.split(':')[0]][0]) 
        df['Name'] = df['Name'].apply(lambda x: [i for i in x.translate(str.maketrans('', '', string.punctuation)).split(' ')]) 
        self.stocks_list = list(set(list(df['Symbol']) + list([item for sublist in list(df['Name']) for item in sublist]))) 
        self.stocks_list = list(map(str.lower,self.stocks_list))
         
    def correct(self, sentence): 
        sentence = sentence.split(' ') 
        corrected = [] 
        for word in sentence:
            if word in self.stocks_list:
                corrected+=[word]
                continue
            try: 
                corrected += [sorted([str(i).split(', ') for i in self.speller.lookup(word, Verbosity.ALL, max_edit_distance=3)], key = lambda x: int(x[2])**(1/(int(x[1])+1)), reverse=True)[0][0]] 
            except: 
                continue 
        return ' '.join(corrected) 
speller = spellchecker()


    

@app.route('/',methods=['GET','POST'])
def index():
    global results
            
    if request.method == "POST": 
        type_of_query = request.form.get("type")
        
        if type_of_query =="refresh":
            try: 
                wrapper(1)
            except:
                return {"Status":"Success"}
            return {"Status":"Success"}
        
        original_query=request.form.get("query")
        no_spell_correct_query=request.form.get("query")
        x= request.form.get("query")
        print("X: ", x)
        print('Original Queried',x)
        x=speller.correct(x)
        x=lem_text(x)
        no_spell_correct_query=lem_text(no_spell_correct_query)
#         print(type_of_query)
        print('Lem Query is',x)
        x=remove_stopwords(x)
        no_spell_correct_query=remove_stopwords(no_spell_correct_query)
        print("Term Queried: ",x)
    
        
        
        
        if type_of_query == "title":

            results = searchfunction(x)
            strlen=getInputLength(x)
            if strlen == 1:
                print(convertToDic(results))
                return(convertToDic(results))

            while len(results)<=10 and strlen>1:
                print('less than 10 results ')
                
#                 strlen=getInputLength(x)
                
                initdebug = convertToDic(results)
   

                if strlen==2:
                    results= dictcombine(initdebug,twosearch(x,strlen))
                    print(results)
                    return (results) 
                elif strlen==3:
                    results = dictcombine(initdebug,threesearch(x,strlen))
                    print(results)
                    return (results) 
                elif strlen == 4:
                    results = dictcombine(initdebug,foursearch(x,strlen))
                    print(results)
                    return (results) 
                elif strlen == 5:
                    results = dictcombine(initdebug,fivesearch(x,strlen))
                    print(results)
                    return (results) 
                else:
                    #UI need add this case
                    return {}
#             print(results)
            return (convertToDic(results))
                
            
#             print(results)        
#             return (results)        
        
# debug = convertToDic(results)

#             print(f'Saw {len(results)} result(s).\n')
#             return convertToDic(results)
#             return(convertToDic(results))

        if type_of_query == "sorted_date_title":
            results = solr.search(f'title:{x}',rows=100,fl=['title','stocks_mentioned','key_points','date_published','url','predict_cat','extract_summarizer'],fq=['',''])
            print(f'Saw {len(results)} result(s).\n')
            print(sortDicbyDate(results))
            return sortDicbyDate(results)
        
        if type_of_query == "sorted_date_ticker":
            results = solr.search(f'stocks_mentioned:{original_query}',rows=100,fl=['title','stocks_mentioned','key_points','date_published','url','predict_cat','extract_summarizer'],fq=['',''])
            print("elif ran")
            print(f'Saw {len(results)} result(s).\n')
            print(sortDicbyDate(results))
            return sortDicbyDate(results)
        
        if type_of_query == "ticker":
            results = solr.search(f'stocks_mentioned:{original_query}',rows=100,fl=['title','stocks_mentioned','key_points','date_published','url','predict_cat','extract_summarizer'],fq=['',''])
            print("elif ran")
            print(f'Saw {len(results)} result(s).\n')
            return convertToDic(results)
        
        if type_of_query == "checker":
            return speller.correct(original_query)
        
        if type_of_query == "sorted_date_title_no_spell":
            results = searchfunction(no_spell_correct_query)
            strlen=getInputLength(no_spell_correct_query)
            if strlen == 1:
                print(sortCombinedDictByDate(convertToDic(results)))
                return(sortCombinedDictByDate(convertToDic(results)))

            while len(results)<=10 and strlen>1:
                print('less than 10 results ')
                
#                 strlen=getInputLength(x)
                
                initdebug = convertToDic(results)
   

                if strlen==2:
                    results= dictcombine(initdebug,twosearch(no_spell_correct_query,strlen))
                    print(results)
                    print(sortCombinedDictByDate(results))
                    return (sortCombinedDictByDate(results)) 
                elif strlen==3:
                    results = dictcombine(initdebug,threesearch(no_spell_correct_query,strlen))
                    print(sortCombinedDictByDate(results))
                    return (sortCombinedDictByDate(results)) 
                elif strlen == 4:
                    results = dictcombine(initdebug,foursearch(no_spell_correct_query,strlen))
                    print(sortCombinedDictByDate(results))
                    return (sortCombinedDictByDate(results)) 
                elif strlen == 5:
                    results = dictcombine(initdebug,fivesearch(no_spell_correct_query,strlen))
                    print(sortCombinedDictByDate(results))
                    return (sortCombinedDictByDate(results)) 
                else:
                    #UI need add this case
                    return {}
#             print(results)
            return sortCombinedDictByDate(convertToDic(results))
            
            
            
#             results = solr.search(f'title:{no_spell_correct_query}',rows=100,fl=['title','stocks_mentioned','key_points','date_published','url','predict_cat','extract_summarizer'],fq=['',''])
#             print(f'Saw {len(results)} result(s).\n')
#             print(sortDicbyDate(results))
#             return sortDicbyDate(results)
        
        if type_of_query =="no_spell_correct_query":
            results = searchfunction(no_spell_correct_query)
            strlen=getInputLength(no_spell_correct_query)
            if strlen == 1:
                print(convertToDic(results))
                return(convertToDic(results))

            while len(results)<=10 and strlen>1:
                print('less than 10 results ')
                
#                 strlen=getInputLength(x)
                
                initdebug = convertToDic(results)
   

                if strlen==2:
                    results= dictcombine(initdebug,twosearch(no_spell_correct_query,strlen))
                    print((results))
                    return ((results)) 
                elif strlen==3:
                    results = dictcombine(initdebug,threesearch(no_spell_correct_query,strlen))
                    print((results))
                    return ((results)) 
                elif strlen == 4:
                    results = dictcombine(initdebug,foursearch(no_spell_correct_query,strlen))
                    print((results))
                    return ((results)) 
                elif strlen == 5:
                    results = dictcombine(initdebug,fivesearch(no_spell_correct_query,strlen))
                    print((results))
                    return ((results)) 
                else:
                    #UI need add this case
                    return {}
#             print(results)
            return (convertToDic(results))
            

    else:
        print('else statement ran')
        return {'ans':[]}


if __name__ == "__main__":
    app.run(port=5003)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


2022-04-10 15:16:15,009 INFO: * Running on http://127.0.0.1:5003/ (Press CTRL+C to quit)
